In [ ]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [4]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [5]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [6]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [7]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_VSP_clusters, \
                     test_size=0.2, stratify=label_material)

    res = leave_one_sample_out(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)

In [8]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])
df.loc['average'] = df.mean()
display(df)

df.to_csv('thrasos/within_trainedonclusters_nearest_neighbor.csv')

,STSIMM,material,visiprog_original,visiprog
0,0.839847,0.938990,0.876072,0.927550
1,0.855100,0.944709,0.860820,0.937083
2,0.863680,0.946616,0.898951,0.949476
3,0.857007,0.939943,0.855100,0.945663
4,0.855100,0.952336,0.876072,0.954242
5,0.860820,0.960915,0.905624,0.951382
6,0.842707,0.955195,0.872259,0.939943
7,0.852240,0.943756,0.888465,0.930410
8,0.845567,0.950429,0.883699,0.948522
9,0.861773,0.951382,0.874166,0.941849


In [ ]:
print(Y_mat_test)